 # Basic Usage of py_SBeLT

## Installation

In [ ]:
!pip install sbelt

Once `sbelt` has been installed, we can import the `sbelt_runner` module. The sbelt_runner module is responsible for executing an instance of an py_SBeLT as it is described in py_SBeLT's [paper.md](https://github.com/szwiep/py_SBeLT/blob/master/paper/paper.md). 

In [ ]:
from sbelt import sbelt_runner

## Parameters and Running

The `sbelt_runner` module contains a function `run` which we will use to execute a run of py_SBeLT. `run` takes 13 parameters/arguments. The project's [API documentation](https://github.com/szwiep/py_SBeLT/tree/master/docs/API) describes these parameters but we also provide the table below:


| Parameter | Type | Tested Range | Default | Description |
| ----------- | ------- | ------ | ----------- | ----------- |
| particle_pack_dens | float | 0.50, 0.80 | 0.78 | The packing fraction of the model particles (-) |
| bed_length | int | 100, 1000 | 100 | Length of the domain in the streamwise direction (mm) |
| particle_diam | int or float =+ 0.5 | 0.5, 10 | 0.5 | Grain diameter (mm) |
| num_subregions | int | 1, 10 | 4 | The number of bed subregions |
| level_limit | int | 1, 3 | 3 | The maximum number of levels permitted (i.e how many particles high to stack)  |
| iterations | int | 1, 1000000 | 1000 | The number of iterations to run |
| poiss_lambda | int | 1, 5 | 5 | Lamba for poisson dist., used to determine the number of entrainment events |
| gauss | boolean | - | False | Flag for which distribution to sample from for hop calculations. True=Normal, False=logNormal |
| gauss_mu | float | 0.25, 2 | 1.0 | Mean/expectation of the logNormal/Normal distribution for hop calculations |
| gauss_sigma | float | 0.25, 0.50 | 0.25 | Standard deviation of logNormal/Normal distribution for hop calculations|
| data_save_interval | int | 1, 2 | 1 | How often to record model particle arrays (e.g 1=save every iteration, 2=save every other) |
| height_dependant_entr | boolean | - | False | Flag indicating whether model automatically entrains particles that are at the height/level limit |
| out_path | string | - | '.' | The location/directory to save model run data |
| out_name | string | - | 'sbelt-out' | Filename for model run data |


If we do not pass any arguments to `run` then the default parameters (described in the [project's DEFAULT_PARAMS.md](https://github.com/szwiep/py_SBeLT/blob/master/docs/DEFAULT_PARAMS.md)) will be used. Let's start by using the default parameters!

### Running with Default Parameters

In [ ]:
sbelt_runner.run()

And that's all it takes! We can see that there is now a file `./sbelt-out.hdf5` located in our directory. This file contains information from the sbelt run including the placement of all particles in the stream for each iteration and metrics such as average age over all particles each iteration. For more detail regarding the information stored and how to interact with it, see the project documentation and additional notebooks.




### Running with User-defined Parameters

But what if we don't want to use the default parameters but instead what to try our own? Let's try!

Instead of using the default parameters, let's execute a run of sbelt over _3000 iterations_, with a _bed length of 150 mm_, _3 subregions_, and the number of entrainment events per-iteration being sample from a poissoin distribution parameterized by _$\lambda$ = 2_. Note that we will also need to create a new filename since we have already written to the default filename (`./sbelt-out.hdf5`) and run will not overwrite model files. Let's use `user-defined-sbelt`.

In [ ]:
sbelt_runner.run(iterations=3000, bed_length=150, num_subregions=3, poiss_lambda=2, out_name='user-defined-sbelt')

Now that we've got two files full of particle-related information, let's plot some of it!

## Plotting

The `sbelt` package comes with some basic plotting logic. We can access these functions with the following import:

In [ ]:
from sbelt.plots import plotting

Each function in `plotting` will require information derived from the `sbelt.hdf5` files (as well a file names and save locations, if desired) which we created in the **Parameters and Running** section. We will need to import both `numpy` and `h5py` into our envrionment to allow us to open/handle the `sbelt.hdf5` files and their stored data structures.

In [ ]:
import numpy as np
import h5py

### Plotting the Stream

Let's start by plotting the stream at iteration 300 for both of our sbelt runs from **Parameters and Running** (default and user-defined parameters). Looking at the [API documentation](https://github.com/szwiep/py_SBeLT/blob/update_docs/docs/API/plotting.html), we can see that `plotting.stream` requires 5 arguments: `iteration`, `bed_particles`, `model_particles`, `x_lim`, and `y_lim`.

For more information regarding the data in the HDF5 file, see the data_storage_sbelt notebook.

In [ ]:
iteration = 300
# y_lim is up to us and how we want the plot to look - not derived from hdf5 file!
y_lim = 10 

with h5py.File('sbelt-out.hdf5', 'r') as f: # open the hdf5 file to read
    # bed particles are stored in the initial_values group
    default_bed_particles = np.array(f['initial_values']['bed']) 
    
    # model particles at the end of iteration i are stored with the key `iteration_i-1`
    default_model_particles_300 = np.array(f['iteration_299']['model'])
    
    # We want to plot the whole stream so let x_lim = length of the bed
    default_x_lim = f['params']['bed_length'][()] 

In [ ]:
plotting.stream(iteration, default_bed_particles, default_model_particles_300, default_x_lim, y_lim)

In [ ]:
# For user-defined (ud) run

with h5py.File('user-defined-sbelt.hdf5', 'r') as f: # open the hdf5 file to read
    # bed particles are stored in the initial_values group
    ud_bed_particles = np.array(f['initial_values']['bed']) 
    
     # model particles at the end of iteration i are stored with the key `iteration_i-1`
    ud_model_particles_300 = np.array(f['iteration_299']['model'])
    
    # We want to plot the whole stream so let x_lim = length of the bed
    ud_x_lim = f['params']['bed_length'][()] 

In [ ]:
plotting.stream(iteration, ud_bed_particles, ud_model_particles_300, ud_x_lim, y_lim)

### Plotting the Downstream Particle Crossings

Next, let's plot a histogram and time-series of the particles crossings at the downstream boundary using the `downstream_boundary_hist` and `downstream_boundary_ts` functions, respectively. We will grab the required information (see [API documentation]()) from the `.hdf5` files similarly to how we did in **Plotting the Stream**.

In [ ]:
# For the default run (change filename to try another run)

with h5py.File('sbelt-out.hdf5', 'r') as f:
    # Find how many subregions there are:
    default_num_subregions = f['params']['num_subregions'][()]
    
    # id of the downstream boundary (final subregion's right boundary) is `subregion_num_subregion-1`
    downstream_key = default_num_subregions - 1 # (because subregions are named 0-(N-1))
    
    # Get the crossings at the final subregion:
    default_particle_crossing_list = np.array(f['final_metrics']['subregions'][f'subregion-{downstream_key}-flux'])
    
    # Total number of iterations is stored in params group
    default_iterations = f['params']['iterations'][()]


In [ ]:
plotting.downstream_boundary_hist(default_particle_crossing_list, default_iterations)

In [ ]:
plotting.downstream_boundary_ts(default_particle_crossing_list, default_iterations, 1)

### Plotting the Downstream Particle Crossings with Age

Finally, let's use the final plotting function provided by `sbelt`, `crossing_info_age`. 

In [ ]:
# For the default run (change filename to try another run)

with h5py.File('sbelt-out.hdf5', 'r') as f:
    # Find how many subregions there are:
    default_num_subregions = f['params']['num_subregions'][()]
    
    # id of the downstream boundary (final subregion's right boundary) is `subregion_num_subregion-1`
    downstream_key = default_num_subregions - 1 
    
    # Get the crossings at the final subregion:
    default_particle_crossing_list = np.array(f['final_metrics']['subregions'][f'subregion-{downstream_key}-flux'])
    
    # Get average age and # of iterations
    default_avg_age = np.array(f['final_metrics']['avg_age'])
    default_iterations = f['params']['iterations'][()]

In [ ]:
plotting.crossing_info_age(default_particle_crossing_list, default_avg_age, default_iterations, 1)